## The Façade pattern

The Façade pattern is designed to provide a simple interface to a complex system of
components. 
The Façade pattern is, in many ways, like the Adapter pattern. The primary
difference is that a Façade tries to abstract a simpler interface out of a complex one,
while an Adapter only tries to map one existing interface to another.


![](uml/facade_pattern.png)

### A Façade example
Combine the two processes to convert `*.uml` files.


[PlantUML](https://plantuml.com)


[Installation](https://plantuml.com/starting)

[classdiagram](https://plantuml.com/class-diagram)

In [1]:
import re
from pathlib import Path
from typing import Iterator, Tuple

class FindUML:
    """Find uml files in sub directories
    
    >>> root = Path().absolute()
    >>> f = FindUML(root)
    >>> list(map(str, next(f.uml_file_iter())))  # doctest: +ELLIPSIS
    ['...uml', '...png']
    """
    def __init__(self, base: Path) -> None:
        self.base = base
        self.start_pattern = re.compile(r"@startuml *(.*)")
        
    def uml_file_iter(self) -> Iterator[tuple[Path, Path]]:
        for source in self.base.glob("**/*.uml"):
            if any(n.startswith(".") for n in source.parts):  # skips .hidden 
                continue
            body = source.read_text()
            for output_name in self.start_pattern.findall(body):
                if output_name:
                    target = source.parent / output_name
                else:
                    target = source.with_suffix(".png")
                yield (
                    source.relative_to(self.base),
                    target.relative_to(self.base)
                )


In [11]:
import subprocess

class PlantUML:
    """Convert *.uml files to png"""

    def __init__(
        self, 
        plantjar: Path = Path("uml") /"plantuml.jar",
    ) -> None:
        self.plantjar = plantjar
        
    def process(self, source: Path) -> None:
#         command = [
#             "java", "-jar",
#             str(self.plantjar), "-progress",
#             str(source)
#         ]
        command = f"java -jar {self.plantjar} -progress {source}"
        print(subprocess.run(command, check=True, shell=True))


In [12]:
class GenerateImages:  # Façade
    """Combine UML finder and convert Class to
    make png images from uml files."""
    
    def __init__(self, base: Path) -> None:
        self.finder = FindUML(base)
        self.painter = PlantUML()
        
    def make_all_images(self) -> None:
        for source, target in self.finder.uml_file_iter():
            if (
                not target.exists()
                or source.stat().st_mtime > target.stat().st_mtime
            ):
                print(f"Processing {source} -> {target}")
                self.painter.process(source)
            else:
                print(f"Skipping {source} -> {target}")
                
    
g = GenerateImages(Path.cwd())
g.make_all_images()

Processing uml/fig.uml -> uml/fig.png


Picked up _JAVA_OPTIONS: -Dawt.useSystemAAFontSettings=on -Dswing.aatext=true
[                              ] 0/1

CompletedProcess(args='java -jar uml/plantuml.jar -progress uml/fig.uml', returncode=0)


                                    [##############################] 1/1

In [33]:
if __name__ == '__main__':        
    import doctest
    import subprocess
    name = "10-The Façade pattern"
    doctest.testmod(verbose=False)
    subprocess.run(f'jupyter nbconvert --to script --output test "{name}"', shell=True)
    std_out = subprocess.run('mypy --strict test.py', capture_output=True, shell=True).stdout
    print(std_out.decode('ascii'))

Success: no issues found in 1 source file

